# Series Temporales

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 10]

In [2]:
data = pd.read_csv('energy.csv',index_col=0)
data.index = pd.to_datetime(data.index)
data.columns = ['produccion de energia']
data.head()

,produccion de energia
DATE,
1939-01-01,3.3842
1939-02-01,3.4100
1939-03-01,3.4875
1939-04-01,3.5133
1939-05-01,3.5133


In [3]:
import plotly.plotly as py
import cufflinks as cf

import pandas as pd

cf.set_config_file(offline=True, theme='ggplot')

data.iplot() 

In [4]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(data, model="multiplicative")

In [5]:
result.observed.iplot() 

In [6]:
result.resid.iplot() 

In [7]:
result.seasonal.iplot() 

In [8]:
result.trend.iplot() 

In [16]:
from pmdarima.arima import auto_arima
stepwise_model = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=3822.068, BIC=3846.376, Fit time=12.596 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=4335.944, BIC=4345.667, Fit time=0.082 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=4167.207, BIC=4186.654, Fit time=2.641 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=3945.584, BIC=3965.031, Fit time=3.702 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=3821.170, BIC=3850.340, Fit time=18.694 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 0, 12); AIC=3999.209, BIC=4023.518, Fit time=12.738 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=3814.587, BIC=3848.619, Fit time=45.131 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=3937.745, BIC=3966.916, Fit time=16.164 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=3812.216, BIC=3851.109, Fit time=35.658 seconds
Fit ARIMA: order=(2, 1

In [17]:
train = data.loc['1985-01-01':'2016-12-01']
test = data.loc['2017-01-01':]

In [18]:
stepwise_model.fit(train)

ARIMA(callback=None, disp=0, maxiter=None, method=None, order=(3, 1, 3),
      out_of_sample_size=0, scoring='mse', scoring_args={},
      seasonal_order=(2, 1, 2, 12), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend=None,
      with_intercept=True)

In [24]:
future_forecast = stepwise_model.predict(n_periods=32)
print(future_forecast)

[121.5870121  109.01412242  99.62785733  89.68270237  90.94640076
 102.92591323 112.63830588 111.8517523  100.51781465  90.97834126
  94.86959609 111.17990255 119.74267969 109.84068251 100.89823422
  89.32374132  91.22256988 102.248152   111.72471972 110.60716171
 100.02149552  90.99322802  95.2696238  108.96889497 119.13993919
 109.88745421 100.40805708  88.97970193  90.73100044 102.02154734
 111.58469409 110.6177252 ]


In [25]:
future_forecast = pd.DataFrame(future_forecast,index = test.index,columns=["Prediction"])
pd.concat([test,future_forecast],axis=1).iplot()

In [26]:
pd.concat([data,future_forecast],axis=1).iplot()